In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import warnings
warnings.filterwarnings('ignore')
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
# Reading the dataset
train_df=pd.read_csv('https://raw.githubusercontent.com/bkorkmaz29/artificial-intelligence/main/Ex5/data/X_train.csv')
train_df_label=pd.read_csv('https://raw.githubusercontent.com/bkorkmaz29/artificial-intelligence/main/Ex5/data/y_train.csv')

test_df=pd.read_csv('https://raw.githubusercontent.com/bkorkmaz29/artificial-intelligence/main/Ex5/data/X_test.csv')
test_df_label=pd.read_csv('https://raw.githubusercontent.com/bkorkmaz29/artificial-intelligence/main/Ex5/data/y_test.csv')

# Normalizing images
def normalize(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label


num_training_examples = len(train_df)
dataset = tf.data.Dataset.from_tensor_slices((train_df.values.reshape(-1, 32, 32,1), train_df_label.to_numpy()))

num_test_examples = len(test_df)
dataset_test = tf.data.Dataset.from_tensor_slices((test_df.values.reshape(-1, 32, 32,1), test_df_label.to_numpy()))

batch_size = 32

training_batches = dataset.cache().shuffle(num_training_examples//4).map(normalize).batch(batch_size).prefetch(1)
testing_batches = dataset.cache().shuffle(num_test_examples//4).map(normalize).batch(batch_size).prefetch(1)


In [ ]:
# Neural network initiation
my_model = tf.keras.Sequential([
           tf.keras.layers.Flatten(input_shape = (32,32,1)),
           tf.keras.layers.Dense(512, activation = 'relu'),
           tf.keras.layers.Dense(256, activation = 'relu'),
           tf.keras.layers.Dense(30, activation = 'softmax')
])

print(my_model.summary())

In [ ]:
# Training

my_model.compile(optimizer='Adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

EPOCHS = 50

early_stop=tf.keras.callbacks.EarlyStopping(
    monitor='loss', patience=2, 
)

history = my_model.fit(training_batches, epochs = EPOCHS, callbacks=[early_stop])

In [ ]:
# Testing the model

for image_batch, label_batch in testing_batches.take(1):
    loss, accuracy = my_model.evaluate(image_batch, label_batch)

print('\nTesting Loss : {:,.3f}'.format(loss))
print(' Testing Accuracy : {:.3%}'.format(accuracy))